In [3]:
import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn.metrics
from sklearn.utils.multiclass import type_of_target

import autosklearn.classification

In [4]:
# import datasets
X_raw = pd.read_csv("dataset_normalised.csv")
y_raw = pd.read_csv("labnf.csv")

# numberical column that are needed for training
numeric_columns=['Shares (Basic)',
       'Shares (Diluted)', 'Cash, Cash Equivalents & Short Term Investments',
       'Accounts & Notes Receivable', 'Inventories', 'Total Current Assets',
       'Property, Plant & Equipment, Net',
       'Long Term Investments & Receivables', 'Other Long Term Assets',
       'Total Noncurrent Assets', 'Total Assets', 'Payables & Accruals',
       'Short Term Debt', 'Total Current Liabilities', 'Long Term Debt',
       'Total Noncurrent Liabilities', 'Total Liabilities',
       'Share Capital & Additional Paid-In Capital', 'Treasury Stock',
       'Retained Earnings', 'Total Equity', 'Total Liabilities & Equity',
       'Price_Present']
X = X_raw[numeric_columns]
y = y_raw["Label"]

In [5]:
type_of_target(y)

'multiclass'

In [6]:
# test-train split
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, random_state=1
)

## AutoML Attempt 1

In [28]:
automl = autosklearn.classification.AutoSklearnClassifier(exclude_estimators=["mlp"])
automl.fit(X_train, y_train)

[WARNING] [2021-04-10 14:52:14,579:Client-AutoMLSMBO(1)::3c2aa69a-99de-11eb-99ba-955994831015] Configuration 114 not found
[WARNING] [2021-04-10 14:52:14,580:Client-AutoMLSMBO(1)::3c2aa69a-99de-11eb-99ba-955994831015] Configuration 338 not found
[WARNING] [2021-04-10 14:52:14,580:Client-AutoMLSMBO(1)::3c2aa69a-99de-11eb-99ba-955994831015] Configuration 43 not found
[WARNING] [2021-04-10 14:52:14,582:Client-AutoMLSMBO(1)::3c2aa69a-99de-11eb-99ba-955994831015] Configuration 283 not found
[WARNING] [2021-04-10 14:52:14,582:Client-AutoMLSMBO(1)::3c2aa69a-99de-11eb-99ba-955994831015] Configuration 576 not found
[WARNING] [2021-04-10 14:52:14,582:Client-AutoMLSMBO(1)::3c2aa69a-99de-11eb-99ba-955994831015] Configuration 71 not found
[WARNING] [2021-04-10 14:52:14,583:Client-AutoMLSMBO(1)::3c2aa69a-99de-11eb-99ba-955994831015] Configuration 162 not found
[WARNING] [2021-04-10 14:52:14,583:Client-AutoMLSMBO(1)::3c2aa69a-99de-11eb-99ba-955994831015] Configuration 67 not found
[WARNING] [2021-04-

AutoSklearnClassifier(exclude_estimators=['mlp'], per_run_time_limit=360)

In [29]:
print(automl.show_models())

[(1.000000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler', 'feature_preprocessor:__choice__': 'select_percentile_classification', 'classifier:random_forest:bootstrap': 'True', 'classifier:random_forest:criterion': 'gini', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.8434454038631288, 'classifier:random_forest:max_leaf_nodes': 'None', 'classifier:random_forest:min_impurity_decrease': 0.0, 'classifier:random_forest:min_samples_leaf': 15, 'classifier:random_forest:min_samples_split': 3, 'classifier:random_forest:min_weight_fraction_leaf': 0.0, 'data_p

In [30]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: 3c2aa69a-99de-11eb-99ba-955994831015
  Metric: accuracy
  Best validation score: 0.487294
  Number of target algorithm runs: 19
  Number of successful target algorithm runs: 2
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 10
  Number of target algorithms that exceeded the memory limit: 6



In [31]:
y_pred = automl.predict(X_test)
print(sklearn.metrics.classification_report(y_test, y_pred,zero_division=False))

              precision    recall  f1-score   support

           0       0.49      0.77      0.60      6803
           1       0.45      0.32      0.37      5152
           2       0.63      0.11      0.19      2752

    accuracy                           0.49     14707
   macro avg       0.52      0.40      0.39     14707
weighted avg       0.50      0.49      0.44     14707



## AutoML Attempt2

In [25]:
automl2 = autosklearn.classification.AutoSklearnClassifier(
        n_jobs=4,
        exclude_estimators=["mlp"],
        memory_limit=3072,
        seed=5,
    )
automl.fit(X_train, y_train, dataset_name='fundamentals1')

# Print statistics about the auto-sklearn run such as number of
# iterations, number of models failed with a time out.
print(automl.sprint_statistics())

/home/alpha91/miniconda3/envs/ds/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44997 instead
  warnings.warn(


auto-sklearn results:
  Dataset name: fundamentals1
  Metric: accuracy
  Best validation score: 0.459066
  Number of target algorithm runs: 17
  Number of successful target algorithm runs: 1
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 16
  Number of target algorithms that exceeded the memory limit: 0



In [26]:
y_pred = automl.predict(X_test)
print(sklearn.metrics.classification_report(y_test, y_pred,zero_division=False))

              precision    recall  f1-score   support

           0       0.46      1.00      0.63      6803
           1       0.00      0.00      0.00      5152
           2       0.00      0.00      0.00      2752

    accuracy                           0.46     14707
   macro avg       0.15      0.33      0.21     14707
weighted avg       0.21      0.46      0.29     14707



In [27]:
print(automl.get_models_with_weights())

AttributeError: 'MyDummyClassifier' object has no attribute 'dataset_properties'